In [ ]:
import numpy as np
from astropy.io import fits
from matplotlib import pyplot as plt
from matplotlib import colors
from photutils import EllipticalAnnulus
import marvin
from marvin.tools.maps import Maps
from marvin.tools.image import Image
import marvin.utils.plot.map as mapplot
from VW_PCA import VW_PCA
from marvin import config

dir = '/data/kate/MaNGA/'
fig_dir = dir+'FIGS/'

# login to receive a token
config.release ='DR15'
#config.login()

In [ ]:
cat = fits.open(dir+'DATA/MPL7/drpall-v2_4_3_gals.fits')
tbdata = cat[1].data
cat.close()

#Remove bad IFUs with critical flags
DRP3QUAL = tbdata.field('DRP3QUAL')
ind_good = np.where((DRP3QUAL < 300) & (tbdata.field('nsa_z') > 0.))

plateifu = tbdata.field('plateifu')[ind_good]
z = tbdata.field('nsa_z')[ind_good]
petro_r_50 = tbdata.field('nsa_elpetro_th50_r')[ind_good]
nsa_elpetro_ba = tbdata.field('nsa_elpetro_ba')[ind_good]
nsa_elpetro_mass = tbdata.field('nsa_elpetro_mass')[ind_good]
nsa_elpetro_phi = tbdata.field('nsa_elpetro_phi')[ind_good]

# Plot PCA classes

In [ ]:
select = np.where( (nsa_elpetro_mass > 1E10) & (z < 0.05) & (petro_r_50 > 5.) )

n_select = 10

# make a color map of fixed colors
cmap = colors.ListedColormap([ 'red', 'blue', 'goldenrod', 'green', 'purple'])
bounds=[1,2,3,4,5,6]
norm = colors.BoundaryNorm(bounds, cmap.N)

cols=4

fig, axs = plt.subplots(nrows=n_select, ncols=cols, figsize=(3*cols, 2.5*n_select))
fig.subplots_adjust(hspace=0.3, wspace=.22)

for i in range(n_select):
    for j in range(2):

        ind = select[0][(2*i+j)]
        
        #Plot SDSS image
        image = Image(plateifu=plateifu[ind])
        img = axs[i, j*2].imshow(image.data)
        axs[i, j*2].set_title(str(plateifu[ind]), fontsize=13)
        axs[i, j*2].get_xaxis().set_visible(False) #Suppress tick labels
        axs[i, j*2].get_yaxis().set_visible(False)

        #Plot PCA class map
        hdu = fits.open(dir+'PCA_results_MPL7_fits/manga-'+plateifu[ind]+'_PCA.fits.gz')
        sfh_class = hdu['classmap'].data
        sfh_class = np.ma.MaskedArray(sfh_class, mask=sfh_class < 1) #Mask junk
        spx_bin_mask = hdu['spx_bin_mask'].data
    
        #Mask bad data
        PCA_SNR_masked = np.ma.MaskedArray(sfh_class, mask=((hdu['snr4000A'].data < 3.) | 
                                 (hdu['PC1'].data < -10.) | (hdu['PC2'].data > 10.)))
        
        sidelen = np.shape(PCA_SNR_masked)[0]
        centre = (sidelen-1) / 2
             
        #Define no coverage (edge of IFU) from Halpha map (can use any map).
        maps = Maps(plateifu=plateifu[ind])
        ha = maps['emline_gflux_ha_6564']
        nocov = ha.mask
        
        #Bad data mask
        bad_data = np.ma.getmaskarray(PCA_SNR_masked) * 2**30 # and set bit 30 (DONOTUSE)
        
        masks = nocov | bad_data
        
        fig, ax, cb = mapplot.plot(value=PCA_SNR_masked, mask=masks, use_masks='NOCOV', \
                     cmap=cmap, fig=fig, ax=axs[i, (j*2)+1], cbrange=(0.5,5.5), return_cb=True)
        cb.remove() #Remove colour bar
   
plt.savefig(fig_dir+'Example_PCA.pdf', bbox_inches='tight')
plt.show()

# Classify spaxels based on PC1 PC2 values (in case you want to change the classification boundaries)

In [ ]:
vwPca = VW_PCA()

vertices_lowmass = {
"psb_cut":0.39, \
"sb_vert1":-2.3, \
"sf_vert1":-5.8, \
"sf_vert2":-5.5, \
"sf_vert3":-2.2, \
"sf_vert4":-3.2, \
"green_vert1": -0.2, \
"green_vert2": -2.1, \
"junk_y_lower": -1.4, \
"junk_y_lower2": -3., \
"junk_y_upper": 2., \
"left_cut": -7.1, \
"right_cut": 2., \
}

vertices_highmass = {
"psb_cut":0.39, \
"sb_vert1":-1.9, \
"sf_vert1":-5.3-0.07, \
"sf_vert2":-5.0-0.07, \
"sf_vert3":-2.2-0.07, \
"sf_vert4":-3.2-0.07, \
"green_vert1": -0.3-0.07, \
"green_vert2": -2.0-0.07, \
"junk_y_lower": -1.2, \
"junk_y_lower2": -3., \
"junk_y_upper": 2., \
"left_cut": -7.1-0.07, \
"right_cut": 2.-0.07, \
}

#Just for 10 galaxies
for i in range(len(select[0]))[0:10]:
    
    ind = select[0][i]

    hdu = fits.open(dir+'PCA_results_MPL7_fits/manga-'+plateifu[ind]+'_PCA.fits.gz')
    pc1 = hdu['PC1'].data
    pc2 = hdu['PC2'].data
    
    #Classification of spaxels
    points = np.stack((pc1.flatten(), pc2.flatten()), axis=1) #Points needs to be a (ngal, 2) array

    #Classification boundaries depend on stellar mass
    if nsa_elpetro_mass[i] < 1E10:
        class_map = vwPca.PCA_classify(points, vertices_lowmass)
    elif nsa_elpetro_mass[i] >= 1E10:
        class_map = vwPca.PCA_classify(points, vertices_highmass)

    class_map_reshaped = class_map.reshape(pc1.shape[0], pc1.shape[1])


# Count spaxels in an annulus/aperture

In [ ]:
#Setting up aperture

select_onegal = np.where(plateifu=='7991-12701')

ind = select_onegal[0][0]

hdu = fits.open(dir+'PCA_results_MPL7_fits/manga-'+plateifu[ind]+'_PCA.fits.gz')

# assume center is the IFU's ra and dec, which is ok for most IFUs. May not be true for mergers and 7443-12701.
x = hdu[1].data.shape[1]/2
y = hdu[1].data.shape[0]/2

spaxel_size = 0.5 # "/pix pixel scale

#Convert from arcsec to spaxels
ellip_ap_radius_pix_in_major = ( petro_r_50[ind] / 2. ) / spaxel_size
ellip_ap_radius_pix_out_major = ( (petro_r_50[ind] / 2.) + 5.  ) / spaxel_size # 5" annulus
ellip_ap_radius_pix_out_minor = nsa_elpetro_ba[ind] * ellip_ap_radius_pix_out_major

#The rotation angle in radians of the semimajor axis from the
#positive x axis. The rotation angle increases counterclockwise.
#nsa_elpetro_phi is Angle (E of N) - needs -90 to make the right way up.
theta = np.radians(nsa_elpetro_phi[ind]-90.)

In [ ]:
#Function to return an elliptical annulus/aperture mask - can also replace with Marvin getAperture
def getAperture(x, y, ellip_ap_radius_pix_in_major, ellip_ap_radius_pix_out_major, ellip_ap_radius_pix_out_minor, \
                theta):

    ellipical_annuli = EllipticalAnnulus([x, y], ellip_ap_radius_pix_in_major, ellip_ap_radius_pix_out_major, \
                                         ellip_ap_radius_pix_out_minor, theta) #In pixels
    masks_ellipical = ellipical_annuli.to_mask(method='exact')
    mask_ellipical = masks_ellipical[0] #Because masks is a list (even if it has one element)
            
    return mask_ellipical

In [ ]:
#Get elliptical annulus/aperture mask
mask_ellipical = getAperture(x, y, ellip_ap_radius_pix_in_major, ellip_ap_radius_pix_out_major, \
                             ellip_ap_radius_pix_out_minor, theta)

In [ ]:
hdu = fits.open(dir+'PCA_results_MPL6_fits/manga-'+plateifu[ind]+'_PCA.fits.gz')
sfh_class = hdu['classmap'].data.T
sfh_class = np.ma.MaskedArray(sfh_class, mask=sfh_class < 0) #Mask junk

#Apply quality mask
PCA_SNR_masked = np.ma.MaskedArray(sfh_class, mask=((hdu['snr4000A'].data < 3.) | 
                                 (hdu['PC1'].data < -10.) | (hdu['PC2'].data > 10.)))

#Apply spx mask so spaxels don't get double counted
PCA_spx_SNR_masked = np.ma.MaskedArray(PCA_SNR_masked, mask=hdu['spx_bin_mask'].data < 1)

#Apply mask to a map
PCA_cutout = mask_ellipical.cutout(PCA_spx_SNR_masked)

In [ ]:
#If not all masked, print total number of spaxels in each PCA class in the annulus
if (PCA_cutout.mask.all() == False):
    total_spaxels_ell = np.ma.sum( mask_ellipical.data[np.ma.where(PCA_cutout > 0)])
    red_spaxels_ell = np.ma.sum( mask_ellipical.data[np.ma.where(PCA_cutout == 1)])
    sf_spaxels_ell = np.ma.sum( mask_ellipical.data[np.ma.where(PCA_cutout == 2)])
    sb_spaxels_ell = np.ma.sum( mask_ellipical.data[np.ma.where(PCA_cutout == 3)])
    green_spaxels_ell = np.ma.sum( mask_ellipical.data[np.ma.where(PCA_cutout == 4)])
    psb_spaxels_ell = np.ma.sum( mask_ellipical.data[np.ma.where(PCA_cutout == 5)])

print(total_spaxels_ell, sf_spaxels_ell)

frac_sf = sf_spaxels_ell / total_spaxels_ell
print(frac_sf)